# PRATIQUES (PARTIE I)
## PHASE 2 : APPLICATIONS SCIENTIFIQUES ET ENTREPRENEURIALES (I)

## Préliminaire

- Commences par décompresser le fichier dump.tar. Tu obtiendra un dossier dump constitué de deux autres dossiers.

- Récupère les données de CDS16 dans Mongodb
mongorestore --uri="mongodb://localhost:27017" dump. Ceci va créer deux bases de données dans MongoDB CDS16 et CDS29. Dans notre cas, nous allons utiliser CDS16.

- pour vérifier, dans le shell de MongoDB, exécutes :

&gt; show dbs ==> Tu verra CDS16 et CDS29

- Pour explorter la base de données vers un fichier csv, dans le shell CMD, exécutes :

&gt; mongoexport --uri="mongodb://localhost:27017" --db=CDS16 --collection=molecules --type=csv --out=molecules.csv --fields=_id, smile, ....

## Importer les modules nécessaires

Nous commençons par importer les bibliothèques à utiliser :

- Pandas
- Quelques Classifieurs : SVM, LogisticRegression et HistGradientBoostingClassifier
- train_test_split : pour diviser le dataset en données d'entraînement et données de test
- Quelques métriques : accuracy, GMEAN, Kappa et ROC

In [219]:
#Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
#Cet algorithme supporte les valeurs nulles
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.metrics import geometric_mean_score #for GMEAN
from sklearn.metrics import cohen_kappa_score #for Kappa
from sklearn.metrics import roc_auc_score #for ROC

## Récupération du dataset

Une fois les modules importés, on commence par récupérer les données à partir du fichiers csv que nous avons obtenu à partir de la base de données MongoDB

In [222]:
molecules = pd.read_csv("molecules.csv")
molecules.shape

(82, 88)

Comme nous pouvons le remarquer, notre dataset est constitué de 82 echantillons. Chaque echantillon est caractérisé de 88 features (colonnes). Pour voir les colonnes, on exécute la commande suivante :

In [224]:
molecules.columns

Index(['_id', 'smiles', 'class', 'MOLECULEID', 'rdmol', 'mfp.bits[0]',
       'mfp.bits[1]', 'mfp.bits[2]', 'mfp.bits[3]', 'mfp.bits[4]',
       'mfp.bits[5]', 'mfp.bits[6]', 'mfp.bits[7]', 'mfp.bits[8]',
       'mfp.bits[9]', 'mfp.bits[10]', 'mfp.bits[11]', 'mfp.bits[12]',
       'mfp.bits[13]', 'mfp.bits[14]', 'mfp.bits[15]', 'mfp.bits[16]',
       'mfp.bits[17]', 'mfp.bits[18]', 'mfp.bits[19]', 'mfp.bits[20]',
       'mfp.bits[21]', 'mfp.bits[22]', 'mfp.bits[23]', 'mfp.bits[24]',
       'mfp.bits[25]', 'mfp.bits[26]', 'mfp.bits[27]', 'mfp.bits[28]',
       'mfp.bits[29]', 'mfp.bits[30]', 'mfp.bits[31]', 'mfp.bits[32]',
       'mfp.bits[33]', 'mfp.bits[34]', 'mfp.bits[35]', 'mfp.bits[36]',
       'mfp.bits[37]', 'mfp.bits[38]', 'mfp.bits[39]', 'mfp.bits[40]',
       'mfp.bits[41]', 'mfp.bits[42]', 'mfp.bits[43]', 'mfp.bits[44]',
       'mfp.bits[45]', 'mfp.bits[46]', 'mfp.bits[47]', 'mfp.bits[48]',
       'mfp.bits[49]', 'mfp.bits[50]', 'mfp.bits[51]', 'mfp.bits[52]',
       'mfp.bit

Pour avoir une idée sur les données, on peut exécuter la commande suivante qui affiche les 5 premières lignes :

In [226]:
molecules.head()

,_id,smiles,class,MOLECULEID,rdmol,mfp.bits[0],mfp.bits[1],mfp.bits[2],mfp.bits[3],mfp.bits[4],...,mfp.bits[73],mfp.bits[74],mfp.bits[75],mfp.bits[76],mfp.bits[77],mfp.bits[78],mfp.bits[79],mfp.bits[80],mfp.bits[81],mfp.count
0,1.0,Nc1nc2c(ncn2C2OC(CO)C(O)C2O)c(=O)[nH]1,Inactive,M18965,776t3gAAAAALAAAAAAAAAAAAAAAUAAAAFgAAAIABCABgAA...,71,75,80,90,128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43
1,2.0,Cc1cn(-c2cc(NC(=O)c3ccc(C)c(Nc4nccc(-c5cccnc5)...,Inactive,M413388,776t3gAAAAALAAAAAAAAAAAAAAAnAAAAKwAAAIABBwBoAA...,19,33,61,64,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69
2,3.0,O=C(O)CC(O)CC(O)C=Cc1c(C2CC2)nc2ccccc2c1-c1ccc...,Inactive,M3412451,776t3gAAAAALAAAAAAAAAAAAAAAfAAAAIgAAAIABCABoAA...,1,25,64,73,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55
3,4.0,O=C1CC(O)CC(C=Cc2c(C3CC3)nc3ccccc3c2-c2ccc(F)c...,Inactive,M97340488,776t3gAAAAALAAAAAAAAAAAAAAAeAAAAIgAAAIABCABgAA...,64,73,80,90,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
4,5.0,Cc1nnc(C(=O)NC(C)(C)c2nc(C(=O)NCc3ccc(F)cc3)c(...,Inactive,M409346,776t3gAAAAALAAAAAAAAAAAAAAAgAAAAIgAAAIABCABoAA...,26,33,80,90,114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57


## Préparation des données

Puisque nous voulons que toutes les colonnes où toutes les valeurs sont ‘0’ peuvent être supprimées, on va afficher ces colonnes :

In [229]:
#Les colonnes contenant seulement les zéros
for col in molecules.columns:
    if molecules[col].isnull().all():
        print(col)

On peut remarquer qu'aucune colonne ne contient que des '0'

Maintenant, nous allons remplacer dans la colonne 'class' la valeur 'Active' par '1' et 'Inactive' par '0'

In [232]:
molecules['class'] = molecules['class'].replace({'Active': 1, 'Inactive': 0})

/var/folders/1j/29251qc56dq87mm8rz7wj6kw0000gn/T/ipykernel_1279/2694727241.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  molecules['class'] = molecules['class'].replace({'Active': 1, 'Inactive': 0})


L'instruction suivante permet de garder seulement les variables prédictives qui représentent l'empreinte moléculaire de chaque composé.

In [234]:
#Garder les variables prédictives qui représentent l'empreinte moléculaire de chaque composé. 
molecules.drop(columns=['_id', 'smiles', 'MOLECULEID', 'rdmol','mfp.count'], inplace=True)

In [235]:
molecules.shape

(82, 83)

La nouvelle structure de notre dataset montre qu'il nous reste seulement 83 features (colonnes)

L'étape suivante est diviser les données (dataset) en une partie représentant le 'Data' et une partie représentant 'Target'

In [238]:
#A partir du dataset séparer les variables prédictives et de la classe
X = molecules.drop('class', axis=1)
y = molecules['class']

On peut afficher quelques données de X et y

In [240]:
X.head()

,mfp.bits[0],mfp.bits[1],mfp.bits[2],mfp.bits[3],mfp.bits[4],mfp.bits[5],mfp.bits[6],mfp.bits[7],mfp.bits[8],mfp.bits[9],...,mfp.bits[72],mfp.bits[73],mfp.bits[74],mfp.bits[75],mfp.bits[76],mfp.bits[77],mfp.bits[78],mfp.bits[79],mfp.bits[80],mfp.bits[81]
0,71,75,80,90,128,147,152,194,210.0,222.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19,33,61,64,84,90,114,128,134.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,25,64,73,80,90,105,117,136.0,169.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64,73,80,90,105,136,175,204,231.0,233.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26,33,80,90,114,121,128,140,163.0,197.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remarque :
Comme on peut le remarquer dans l'affichage précédent, il y a des colonnes qui contient des valeurs manquantes (NaN). Pour avoir une idée sur ces colonnes, on peut exécuter ce script :

In [242]:
X.isna().sum()

mfp.bits[0]      0
mfp.bits[1]      0
mfp.bits[2]      0
mfp.bits[3]      0
mfp.bits[4]      0
                ..
mfp.bits[77]    80
mfp.bits[78]    80
mfp.bits[79]    80
mfp.bits[80]    81
mfp.bits[81]    81
Length: 82, dtype: int64

Dans le résultat précédent, on voit que la colonne 'mfp.bits[77]' contient 80 valeurs manquantes.

In [244]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

## Entrainement et test de la machine

Une fois que nous avons préparé les données, on va diviser les données en une partie qui sera utilisée dans l'entraînement et partie pour le test

In [247]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, train_size=.8, shuffle=True)
Xtrain.shape,Xtest.shape

((65, 82), (17, 82))

Pour l'entraînement de la machine, nous avons deux types d'algorithmes :

- ceux qui supportent le valeurs NaN dans le dataset : HistGradientBoosting, 
- ceux ne supportent pas de valeurs NaN : LogisticRegression, SVM, KNN, ...

### Classification avec HistGradientBoosting

Nous allons commencer par HistGradientBoosting. Cet algorithme supporte les valeurs NaN. Donc on n'a besoin d'aucun traitement supplémentaire sur notre dataset :

In [251]:
model1 = HistGradientBoostingClassifier()
model1.fit(Xtrain, ytrain)  # X_train peut contenir des NaN

HistGradientBoostingClassifier()

Une fois que nous avons entraîner notre modèle, on peut le tester pour voir sa performance 

In [253]:
ypred = model1.predict(Xtest)

In [254]:
print("Accuracy :",accuracy_score(ytest, ypred))

Accuracy : 0.6470588235294118


Le GMEAN est souvent utilisé dans les problèmes de classification avec un dasaet désequilibré (unbalanced dataset), où l'accuracy traditionnelle peut être trompeuse. Il permet de s'assurer que le modèle ne favorise pas une classe au détriment de l'autre.

In [256]:
# Calcul de GMean
gmean = geometric_mean_score(ytest, ypred)
print("GMean :", gmean)

GMean : 0.6324555320336758


En ce qui concerne le score de Kappa :

- < 0	Désaccord total

- 0 – 0.20	Accord faible

- 0.21 – 0.40	Accord moyen

- 0.41 – 0.60	Accord modéré

- 0.61 – 0.80	Accord fort

- 0.81 – 1.00	Accord presque parfait

In [258]:
# Calcul de Kappa
kappa = cohen_kappa_score(ytest, ypred)
print("Kappa de Cohen :", kappa)

Kappa de Cohen : 0.27142857142857135


AUC (Area Under Curve) : aire sous cette courbe ROC, variant entre 0 et 1 :

En général :

- AUC > 0.9 : Excellent.

- AUC entre 0.8 et 0.9 : Bon.

- AUC entre 0.7 et 0.8 : Acceptable.

- AUC entre 0.5 et 0.7 : Faible.

- AUC = 0.5 : Inutile.

In [260]:
# Prédiction des probabilités
yscores = model1.predict_proba(Xtest)[:, 1]  # On prend la colonne des probabilités pour la classe 1

# Calcul de AUROC
auc = roc_auc_score(ytest, yscores)
print("AUC-ROC :", auc)

AUC-ROC : 0.5642857142857143


### Classification avec LogisticRegression

La Régression Logstique n'accepte pas les valeurs manquantes (NaN). Pour remédier à ce problème, nous avons plusieurs solutions, par exemple :

- Solution 1 : Supprimer les lignes contenant des NaN

- Solution 2 : Remplacer les NaN par une valeur appropriée (0, moyenne, valeur la plus fréquente, ...)

Nous allons choisir de remplacer les valeurs manquantes par la moyenne

In [264]:
X.fillna(molecules.mean(), inplace=True)

Essayons de tester que le dataset ne contient plus de valeurs NaN

In [320]:
#Le nombre de valeurs NaN
X.isna().sum().sum()

0

In [267]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, train_size=.8, shuffle=True)
Xtrain.shape,Xtest.shape

((65, 82), (17, 82))

In [268]:
model2 = LogisticRegression()

In [269]:
ypred = model1.predict(Xtest)

In [270]:
print("Accuracy:",accuracy_score(ytest, ypred))

Accuracy: 0.9411764705882353


In [271]:
# Calcul de GMean
gmean = geometric_mean_score(ytest, ypred)
print("GMean :", gmean)

GMean : 0.9486832980505138


In [272]:
# Calcul de Kappa
kappa = cohen_kappa_score(ytest, ypred)
print("Kappa de Cohen :", kappa)

Kappa de Cohen : 0.8811188811188811


In [273]:
# Prédiction des probabilités
yscores = model1.predict_proba(Xtest)[:, 1]  # On prend la colonne des probabilités pour la classe 1

# Calcul de AUROC
auc = roc_auc_score(ytest, yscores)
print("AUC-ROC :", auc)

AUC-ROC : 0.9285714285714286
